In [5]:
import json
# from coreference_resolution import resolve_coreference
from utils.parse_expr import get_template_and_replacements, get_simple_question_type

exp_name = 'memory_core_1'

wikidata_mid_to_fn = json.load(open('data/wikidata_mid_to_fn.json', 'r'))
test_data = json.load(open('data/processed_spice_data/dev_each_type_50.json', 'r'))
# test_data = json.load(open('output/dev_each_type_50/data.json', 'r'))

def sub_mid_to_fn(expression):
    seg_list = expression.split()
    for i in range(len(seg_list)):
        token = seg_list[i].strip(')(')
        if token.startswith('P') or token.startswith('Q'):
            fn = wikidata_mid_to_fn.get(token, "unknown_entity")
            seg_list[i] = seg_list[i].replace(token, fn)
    new_expression = ' '.join(seg_list)
    return new_expression

In [6]:
for qa in test_data:
    # qa['coreference_resolved_question'] = resolve_coreference(qa['question'])
    qa['s_expression_fn'] = sub_mid_to_fn(qa['s_expression'])
    qa['s_expression_cores_fn'] = [sub_mid_to_fn(core) for core in qa['s_expression_cores']]
    template, replacements = get_template_and_replacements(qa['s_expression'], qa['s_expression_cores'])
    qa['template'] = template
    qa['replacements'] = {"variables": {var: qa['s_expression_cores_fn'][replacements['variables'][var]] for var in replacements['variables']}, "constants": replacements['constants'], "functions": replacements['functions']}
    qa['simple_question_type'] = get_simple_question_type(template)

In [7]:
s_expression_cores_examples = json.load(open('data/example/s_expression_cores.json', 'r'))

examples = []
for qa in s_expression_cores_examples:
    examples.append({'role': 'user', 'content': qa['coreference_resolved_question']})
    examples.append({'role': 'assistant', 'content': '\u001F'.join(qa['s_expression_cores_fn'])})
examples

[{'role': 'user',
  'content': 'Which male person was the parent of Ludovico II, Marquess of Saluzzo ?'},
 {'role': 'assistant',
  'content': '(AND (JOIN (R father) Ludovico_II,_Marquess_of_Saluzzo) (JOIN instance_of common_name))'},
 {'role': 'user', 'content': 'Who starred in Pardonnez-moi ?'},
 {'role': 'assistant',
  'content': '(AND (JOIN (R cast_member) Pardonnez-moi) (JOIN instance_of common_name))'},
 {'role': 'user',
  'content': 'What are the job of Christian Ernst von Windheim ?'},
 {'role': 'assistant',
  'content': '(AND (JOIN (R occupation) Christian_Ernst_von_Windheim) (JOIN instance_of occupation))'},
 {'role': 'user', 'content': 'What is the sex of Kenneth Halverson ?'},
 {'role': 'assistant',
  'content': '(AND (JOIN (R sex_or_gender) Kenneth_Halverson) (JOIN instance_of sex))'},
 {'role': 'user',
  'content': 'Which nucleic acid sequence encodes acylglycerol lipase YKL094W ?'},
 {'role': 'assistant',
  'content': '(AND (JOIN (R encoded_by) acylglycerol_lipase_YKL094W

In [8]:
from utils.llm_call import LLM_Call

API_KEYS = ['sk-54964e5c3b8c4998a74f7d3e35b618ac', 'sk-785362add9834d1da5907f4621dc90a8']
API_URL = 'https://api.deepseek.com'
API_MODEL = 'deepseek-chat'

api_pool = [(k, API_URL, API_MODEL) for k in API_KEYS]

LLM = LLM_Call(api_pool=api_pool, openai_params = {'temperature': 0, 'max_tokens': 512})

batch_size = 20
batches = [test_data[i:i + batch_size] for i in range(0, len(test_data), batch_size)]

prompt = """Please strictly follow these rules to convert natural language questions into Lisp-style knowledge graph query expressions without adding explanations:

1. Context handling:
- Entity format: convert spaces to underscores

2. Atomic query construction:
- Forward relation: (JOIN predicate object)
"Who invented the piano?" → (JOIN discoverer_or_inventor Piano)
- Reverse relation: (JOIN (R predicate) subject)
"What is Y's occupation?" → (JOIN (R occupation) Y)
- Existence check: (IS_TRUE subject predicate object)
"Does X depict Y?" → (IS_TRUE X depicts Y)

3. Logical combinations:
- Multi-condition intersection: (AND q1 q2 ...)
"Men who play piano" → (AND (JOIN instrument Piano) (JOIN sex Male))
- Multi-value enumeration: (VALUES v1 v2 ...)
"City A or B" → (VALUES City_A City_B)

4. Special rules:
- Comparisons/counting only extract compared/counted objects:
"Species depicted in most artworks" → (JOIN expressed_in Artwork_series)
- Add type constraints:
"Which occupation is X" → (AND X (JOIN instance_of occupation))
- Output expressions joined with \u001F
"""
for i, batch in enumerate(batches):
    print(f"batch: {i + 1}/{len(batches)}")
    messages_list = []
    for qa in batch:
        messages_list.append([{'role': 'system', 'content': prompt}] + examples + [{'role': 'user', 'content': qa['coreference_resolved_question']}])

    resp_list = await LLM._batch_generate_async(messages_list)

    for j, qa in enumerate(batch):
        resp = resp_list[j]
        qa['predicted_cores'] = [core for core in resp.choices[0].message.content.split('\u001F')]

[('sk-54964e5c3b8c4998a74f7d3e35b618ac', 'https://api.deepseek.com', 'deepseek-chat'), ('sk-785362add9834d1da5907f4621dc90a8', 'https://api.deepseek.com', 'deepseek-chat')]
batch: 1/190


100%|██████████| 20/20 [03:31<00:00, 10.56s/it]


batch: 2/190


100%|██████████| 20/20 [01:11<00:00,  3.58s/it]


batch: 3/190


...

100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


batch: 189/190


100%|██████████| 20/20 [00:15<00:00,  1.30it/s]


batch: 190/190


100%|██████████| 11/11 [00:15<00:00,  1.43s/it]


In [11]:
import os
os.makedirs(f'output/{exp_name}', exist_ok=True)
json.dump(test_data, open(f'output/{exp_name}/cores.json', 'w'), indent=2)